## Text Processing

In [1]:
import pandas as pd
import numpy as np
import string 
string.punctuation
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

Importing my data 

In [2]:
text = pd.read_csv("dfht_25-11_13-02.csv")

In [3]:
text.head()

,Unnamed: 0,Unnamed: 0.1,text,name,year,departure,arrival,theme,latitude,longitude,...,url,arrival_1,arrival_2,arrival_3,theme_1,theme_2,date_slavery_from,date_slavery_to,bought_sold_for,any_money_mention
0,0,0,My name is Vi. And I am 28 years old. I arrive...,Vi,2001.0,Vietnam,American Samoa,Forced labour,-14.354000,-170.773600,...,http://antislavery.ac.uk/items/show/7,American Samoa,NaN,NaN,Forced labour,NaN,NaN,NaN,[],"[('408', '$'), ('4000', '$'), ('2000', '$'), (..."
1,1,1,Thank you for the opportunity to testify befor...,Beatrice,2005.0,Sri Lank,Lebanon,Domestic slavery,33.900000,35.500000,...,http://antislavery.ac.uk/items/show/8,Lebanon,NaN,NaN,Domestic slavery,NaN,NaN,NaN,[],[]
2,2,2,My sister and I were living in a village. We w...,Kavita,2004.0,Sri Lank,India,Domestic slavery,24.071318,79.652396,...,http://antislavery.ac.uk/items/show/102,India,NaN,NaN,Domestic slavery,NaN,2002.0,2004.0,[],[]
3,3,3,I grew up in a village of between 500 and 700 ...,Maria,2005.0,Mexico,United States,Domestic slavery,40.412044,-96.870925,...,http://antislavery.ac.uk/items/show/103,United States,NaN,NaN,Domestic slavery,NaN,1976.0,1981.0,[],"[('200', '$')]"
4,4,4,"I was living in a village, Karko, in the Nuba ...",Mende,2003.0,Sudan,"['Sudan ', 'United Kingdom ']",War slavery,16.865226,32.271106,...,http://antislavery.ac.uk/items/show/104,Sudan,United Kingdom,NaN,War slavery,NaN,1994.0,2000.0,[],[]


In [4]:
text.shape

(1358, 100)

In [5]:
import string 

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def punctuation(n):
    for punctuation in string.punctuation:
        n = n.replace(punctuation, '')
    return n

def lowercase(n):
    n = n.lower()
    return n

def remove_numbers(n):
    n = ''.join(word for word in n if not word.isdigit())
    return n

def remove_stopwords(n):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(n) 
    n = [word for word in word_tokens if not word in stop_words]
    return n

def lemmatize(n):
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in n]
    n = ' '.join(lemmatized)
    return n


In [11]:
text['clean_text'] = text['text'].map(punctuation)
text['clean_text'] = text['clean_text'].map(lowercase)
text['clean_text'] = text['clean_text'].map(remove_numbers)
text['clean_text'] = text['clean_text'].map(remove_stopwords)
text['clean_text'] = text['clean_text'].map(lemmatize)

## LatentDirichletAllocation model

In [12]:
# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer().fit(text['clean_text'])

# data_vectorized = vectorizer.transform(text['clean_text'])

# lda_model = LatentDirichletAllocation(n_components=10).fit(data_vectorized)

# def print_topics(model, vectorizer):
#     for idx, topic in enumerate(model.components_):
#         print("Topic %d:" % (idx))
#         print([(vectorizer.get_feature_names_out()[i], topic[i])
#                         for i in topic.argsort()[:-10 - 1:-1]])
        

# print_topics(lda_model, vectorizer)

three words that appear always together

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(text['clean_text'])

X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
new = pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names())

/Users/paloma/.pyenv/versions/3.8.12/envs/ht_project/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## N-grams

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
tf_idf_vectorizer = CountVectorizer(ngram_range = (3,3))

X = tf_idf_vectorizer.fit_transform(text['clean_text'])

X.toarray()

three_words=pd.DataFrame(X.toarray(),columns = tf_idf_vectorizer.get_feature_names())

In [16]:
# three_words.T.reset_index().groupby('index').sum()

In [17]:
def get_top_3_words(corpus, n=None):
   
    vec = CountVectorizer(ngram_range = (3,3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def get_top_4_words(corpus, n=None):
   
    vec = CountVectorizer(ngram_range = (4,4)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def get_top_5_words(corpus, n=None):
   
    vec = CountVectorizer(ngram_range = (5,5)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def get_top_6_words(corpus, n=None):
   
    vec = CountVectorizer(ngram_range = (6,6)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [18]:
get_top_3_words(text["clean_text"], n=None)

[('human right watch', 167),
 ('original narrative found', 110),
 ('provided human right', 95),
 ('right watch report', 86),
 ('narrative provided human', 80),
 ('name given narrative', 71),
 ('go back home', 54),
 ('narrative provided amnesty', 54),
 ('provided amnesty international', 54),
 ('narrative provided venkatarangaiya', 48),
 ('provided venkatarangaiya foundation', 48),
 ('venkatarangaiya foundation report', 48),
 ('foundation report and', 48),
 ('report and never', 48),
 ('and never lived', 48),
 ('never lived happily', 48),
 ('lived happily ever', 48),
 ('happily ever battle', 48),
 ('ever battle justice', 48),
 ('battle justice go', 48),
 ('justice go voice', 48),
 ('go voice married', 48),
 ('voice married girl', 48),
 ('married girl telangana', 48),
 ('nonprofit organization dedicated', 46),
 ('preventing human trafficking', 46),
 ('told documentary maker', 45),
 ('narrative source youth', 45),
 ('source youth underground', 45),
 ('youth underground nonprofit', 45),
 ('u

In [19]:
get_top_4_words(text["clean_text"], n=None)

[('human right watch report', 86),
 ('provided human right watch', 85),
 ('narrative provided human right', 80),
 ('narrative provided amnesty international', 54),
 ('narrative provided venkatarangaiya foundation', 48),
 ('provided venkatarangaiya foundation report', 48),
 ('venkatarangaiya foundation report and', 48),
 ('foundation report and never', 48),
 ('report and never lived', 48),
 ('and never lived happily', 48),
 ('never lived happily ever', 48),
 ('lived happily ever battle', 48),
 ('happily ever battle justice', 48),
 ('ever battle justice go', 48),
 ('battle justice go voice', 48),
 ('justice go voice married', 48),
 ('go voice married girl', 48),
 ('voice married girl telangana', 48),
 ('narrative source youth underground', 45),
 ('source youth underground nonprofit', 45),
 ('youth underground nonprofit organization', 45),
 ('underground nonprofit organization dedicated', 45),
 ('nonprofit organization dedicated preventing', 45),
 ('organization dedicated preventing human

In [20]:
get_top_5_words(text["clean_text"], n=None)

[('narrative provided human right watch', 70),
 ('provided human right watch report', 54),
 ('narrative provided venkatarangaiya foundation report', 48),
 ('provided venkatarangaiya foundation report and', 48),
 ('venkatarangaiya foundation report and never', 48),
 ('foundation report and never lived', 48),
 ('report and never lived happily', 48),
 ('and never lived happily ever', 48),
 ('never lived happily ever battle', 48),
 ('lived happily ever battle justice', 48),
 ('happily ever battle justice go', 48),
 ('ever battle justice go voice', 48),
 ('battle justice go voice married', 48),
 ('justice go voice married girl', 48),
 ('go voice married girl telangana', 48),
 ('narrative source youth underground nonprofit', 45),
 ('source youth underground nonprofit organization', 45),
 ('youth underground nonprofit organization dedicated', 45),
 ('underground nonprofit organization dedicated preventing', 45),
 ('nonprofit organization dedicated preventing human', 45),
 ('organization dedic

In [21]:
get_top_6_words(text["clean_text"], n=None)

[('narrative provided human right watch report', 54),
 ('narrative provided venkatarangaiya foundation report and', 48),
 ('provided venkatarangaiya foundation report and never', 48),
 ('venkatarangaiya foundation report and never lived', 48),
 ('foundation report and never lived happily', 48),
 ('report and never lived happily ever', 48),
 ('and never lived happily ever battle', 48),
 ('never lived happily ever battle justice', 48),
 ('lived happily ever battle justice go', 48),
 ('happily ever battle justice go voice', 48),
 ('ever battle justice go voice married', 48),
 ('battle justice go voice married girl', 48),
 ('justice go voice married girl telangana', 48),
 ('narrative source youth underground nonprofit organization', 45),
 ('source youth underground nonprofit organization dedicated', 45),
 ('youth underground nonprofit organization dedicated preventing', 45),
 ('underground nonprofit organization dedicated preventing human', 45),
 ('nonprofit organization dedicated preventi